# Projeto

Neste projeto, vamos usar classificadores para identificar *quais são os fatores de risco para o acidente vascular cerebral (AVC)*. Um AVC (*stroke*) é um dano cerebral causado pela interrupção de seu fluxo sanguíneo. Ter um AVC é um problema sério porque pode levar a sequelas, então é melhor preveni-lo do que tentar lidar com suas consequências.

Temos à nossa disposição um conjunto de dados para [predição de AVCs](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset). O que faremos é:

1. Treinar um classificador para predizer se houve ou não houve AVCs
1. Verificar a acurácia do classificador
1. Identificar quais são os fatores que mais provavelmente estão ligados a ter AVCs

## Teoria

### Separando linhas de dados em treino e teste

O que faremos será avaliar um classificador. Lembre-se que para fazer o classificador tivemos que usar uma base de dados pré-classificada. Porém, o que gostaríamos de saber é se, ao ser colocado em produção, o classificador terá um bom desempenho.

Para isso, vamos partir de uma premissa (vou chamar de *premissa número 1* mas esse não é um nome oficial) que é muito importante: a de que nossa base de dados é composta de elementos amostrados aleatoriamente do conjunto total de dados. Nessas condições, a distribuição dos dados na nossa base de dados é a mesma distribuição dos dados que encontraremos no mundo real.

Claro que gostaríamos de saber como nosso classificador se comporta no mundo real, e não na amostra dele que compõe nossa base de dados! Então, precisamos ao menos de ter alguns dados que seguem a mesma distribuição dos dados do mundo real, e que serão usados nas mesmas condições que os dados do mundo real.

Por isso, dividimos nosso conjunto de dados em dois conjuntos: treino e teste. O conjunto de treino é o que será usado para treinar (`fit`) os nossos classificadores. O conjunto de teste será usado para testar (`score`) os classificadores. O conjunto de teste existe para simular uma situação de produção (como se o classificador estivesse operando), então ele não pode ser usado em hipótese alguma para nenhum tipo de ajuste de parâmetros. Ao mesmo tempo, o conjunto de treino é aquele que foi usado para "ensinar" rótulos corretos aos classificadores, então o desempenho dos classificadores no conjunto de treino não é representativo de seu desempenho em produção.

Para dividir os conjuntos de treino e teste, você pode usar a função `train_test_split` do scikit-learn:

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)

### Analisando os dados

Antes de começar qualquer coisa, vamos (como sempre) olhar para os dados que temos. Na nossa tabela, temos uma coluna com a identificação do paciente - essa é uma ID única, anônima, que permite referenciar ao paciente. Também, temos várias colunas com características do paciente: idade, sexo, se fuma, etc. Por fim, temos uma coluna com o *outcome*, que é a coluna `stroke`. Para usar corretamente os classificadores do scikit-learn, você deve separar suas tabelas entre as que têm dados de "entrada" (as features) das que têm dados de "saída" (os *outcomes*).

Outra coisa importante é que os dados deste dataset podem ser categóricos, e nossos sistemas não lidam bem com eles. Por isso, precisamos convertê-los para dados numéricos, por exemplo usando one-hot encoding. Pandas implementa a conversão para one-hot encoding usando `pd.get_dummies(df)`.

In [53]:
import pandas as pd

In [54]:
df = pd.read_csv('data.csv')
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [59]:
pd.get_dummies(df)

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.600000,1,False,True,False,False,...,False,True,False,False,False,True,False,True,False,False
1,61.0,0,0,202.21,28.893237,1,True,False,False,False,...,False,False,True,False,True,False,False,False,True,False
2,80.0,0,1,105.92,32.500000,1,False,True,False,False,...,False,True,False,False,True,False,False,False,True,False
3,49.0,0,0,171.23,34.400000,1,True,False,False,False,...,False,True,False,False,False,True,False,False,False,True
4,79.0,1,0,174.12,24.000000,1,True,False,False,False,...,False,False,True,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,80.0,1,0,83.75,28.893237,0,True,False,False,False,...,False,True,False,False,False,True,False,False,True,False
5106,81.0,0,0,125.20,40.000000,0,True,False,False,False,...,False,False,True,False,False,True,False,False,True,False
5107,35.0,0,0,82.99,30.600000,0,True,False,False,False,...,False,False,True,False,True,False,False,False,True,False
5108,51.0,0,0,166.29,25.600000,0,False,True,False,False,...,False,True,False,False,True,False,False,True,False,False


In [56]:
# removing id column, because it is not useful for the model
df = df.drop(columns=['id'])
# replacing nan values with the mean of the column
for col in df.columns:
    df[col] = df[col].fillna(df[col].mean())

Linear Classification Model

In [57]:
import numpy as np


In [58]:
import autograd.numpy as np_   # Thinly-wrapped version of Numpy
from autograd import grad

def loss( parametros ):
    w, b, pontos, real_value = parametros
    prediction = w.T @ pontos + b
    mse = np_.mean( (prediction - real_value)**2)
    return mse

g = grad(loss)

data = df.drop(columns=["stroke"]).to_numpy()
print(data)
target = df["stroke"].to_numpy()

w = np.random.randn(data.shape[1],1)
b = 0.0
alpha = 10**-3

for n in range(1000):
    grad_ = g( (w, b, data, target) )
    w -= alpha*grad_[0]
    b -= alpha*grad_[1]

print(w)
print(b)

[[67.0 0 1 ... True False False]
 [61.0 0 0 ... False True False]
 [80.0 0 1 ... False True False]
 ...
 [35.0 0 0 ... False True False]
 [51.0 0 0 ... True False False]
 [44.0 0 0 ... False False False]]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 5110 is different from 21)

Modelo de classificação DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# dataset 
# data, all columns except stroke
X = df.drop(columns=["stroke"])

# target, only stroke column
Y = df["stroke"]

# split dataset into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)


clf = DecisionTreeClassifier()

clf.fit(X_train, Y_train)

print(f"Accuracy: {clf.score(X_test, Y_test)}")

importance = []
for i in range(len(clf.feature_importances_)):
   importance.append((X.columns[i],clf.feature_importances_[i]))

importance.sort()
print( importance)


Accuracy: 0.9099804305283757
[('Residence_type_Rural', 0.03488261462603799), ('Residence_type_Urban', 0.014563491606370861), ('age', 0.1945917468613796), ('avg_glucose_level', 0.30539932951000476), ('bmi', 0.24681059493799173), ('ever_married_No', 0.0064302841796899735), ('ever_married_Yes', 0.011739140853798015), ('gender_Female', 0.0022910706818843793), ('gender_Male', 0.023741429958818335), ('gender_Other', 0.0), ('heart_disease', 0.00627777985587675), ('hypertension', 0.021580509563997964), ('smoking_status_Unknown', 0.02045127007909762), ('smoking_status_formerly smoked', 0.03342178735799505), ('smoking_status_never smoked', 0.014467074944118433), ('smoking_status_smokes', 0.013066422580311272), ('work_type_Govt_job', 0.014494117218739251), ('work_type_Never_worked', 0.0), ('work_type_Private', 0.013525671900646305), ('work_type_Self-employed', 0.022265663283241553), ('work_type_children', 0.0)]
